In [1]:
import json
import re
import nltk
import numpy as np

In [2]:
with open("CS5785-final-data/captions.json") as json_file:
    captions_json = json.load(json_file)


In [3]:
train_labels = np.genfromtxt("CS5785-final-data/train.txt",dtype=None)
train_labels = train_labels[:,1]
labels_200 = np.array(list(set(train_labels)))

['yard' 'hill' 'ranch_house' ..., 'television_studio' 'street' 'bayou']


In [4]:
#extract all nouns for each image
wnl = nltk.stem.WordNetLemmatizer()
def extract_noun(sentence):
    #lower case
    lower = sentence.lower()
    #split
    text = nltk.tokenize.word_tokenize(lower)
    #tag
    tagged_text = nltk.pos_tag(text)
    #extraxt noun
    nouns = [w[0] for w in tagged_text if ((w[1]=='NN')|(w[1]=='NNS'))]
    #singular form
    nouns = [wnl.lemmatize(word,'n') for word in nouns]
    return nouns

jpg_nouns = []
for jpg in captions_json.values():
    result = [extract_noun(s) for s in jpg]
    nouns = list(set([item for sublist in result for item in sublist]))
    jpg_nouns.append(nouns)
unique_nouns = list(set([item for sublist in jpg_nouns for item in sublist]))

#extract nouns matching labels_200 for each image
jpg_matched_words_10k = []
for nouns in jpg_nouns:
    #a is of length nouns
    a =[(w in nouns) for w in labels_200]
    matched_labels = []
    if sum(a)>0:
        matched_labels = labels_200[np.array(a)]
    jpg_matched_words_10k.append(matched_labels)

In [15]:
#phrases in labels_200
phrases = [w for w in labels_200 if re.search("_",w)]
phrases = [re.sub("_"," ",w) for w in phrases]

#images captions that include the phrases
def match_phrase(sentence):
    #lower case
    lower = sentence.lower()
    #regular expression to match phrase
    matched_phrase = []
    for ph in phrases:
        if re.match(ph,lower):
            matched_phrase.append(ph)
    return matched_phrase

jpg_matched_phrases_10k = []
for jpg in captions_json.values():    
    result = [match_phrase(s) for s in jpg]
    phrase = list(set([item for sublist in result for item in sublist]))
    jpg_matched_phrases_10k.append(phrase)

In [36]:
#indices of the images and corresponding words
indices_words = [i for i, x in enumerate(jpg_matched_words_10k) if x != []]
words = [jpg_matched_words_10k[i] for i in indices_words]
#indices of of data and corresponding phrase
indices_phrases = [i for i, x in enumerate(jpg_matched_phrases_10k) if x != []]
phrases = [jpg_matched_phrases_10k[i] for i in indices_phrases]

#change everything to np array
indices_words = np.array(indices_words)
indices_phrases = np.array(indices_phrases)
wwords = np.array(words)
phrases = np.array(phrases)


In [ ]:
print indices_words
print words
print indices_phrases
print phrases

In [22]:
#generate the training data for each
tenk_cnn = np.load("CS5785-final-data/alexnet_feat_10k.npy")
tenk_bow = np.load("CS5785-final-data/SIFTBoW_10k.npy")


In [54]:
#create features for tenk_cnn and tenk_bow with word labels
#if a point have two labels, create two points with different labels

tenk_cnn_words = tenk_cnn[indices_words]
tenk_bow_words = tenk_bow[indices_words]

rep = [len(k) for k in words]
tenk_cnn_words_expand = np.repeat(tenk_cnn_words, rep, axis=0)
tenk_bow_words_expand = np.repeat(tenk_bow_words, rep, axis=0)

words_labels = [item for sublist in words for item in sublist]


In [78]:
#add the three phrases
print phrases
phrases_labels = ["sushi_bar","construction_site","baseball_field"]

tenk_cnn_phr = tenk_cnn[indices_phrases]
tenk_bow_phr = tenk_bow[indices_phrases]

tenk_feat_cnn_labelled = np.vstack((tenk_cnn_words_expand,tenk_cnn_phr))
tenk_feat_bow_labelled = np.vstack((tenk_bow_words_expand,tenk_bow_phr))

tenk_labels = words_labels + phrases_labels

[['sushi bar']
 ['construction site']
 ['baseball field']]


In [81]:
print tenk_feat_cnn_labelled.shape
print tenk_feat_bow_labelled.shape
print len(tenk_labels)
np.save("tenk_feat_cnn_labelled.npy",tenk_feat_cnn_labelled)
np.save("tenk_feat_bow_labelled.npy",tenk_feat_bow_labelled)
np.save("tenk_labels.npy",tenk_labels)


(5247L, 4096L)
(5247L, 4096L)
5247
